In [ ]:
import requests
import time
import os
import numpy as np
import subprocess
from operator import itemgetter
import glob
import random
import fnmatch
import itertools
from bs4 import BeautifulSoup
from urllib.request import urlopen

epc_pathname = '/media/smcl/Audio/epc_pages/writing.upenn.edu_08_06_2018_151036/'

os.chdir(epc_pathname)

In [ ]:
def find_paths(dir_path, extension_list):
    media_paths = []
    for root, dirnames, filenames in os.walk(dir_path):
        for filename in fnmatch.filter(filenames, '*'):
            media_paths.append(os.path.join(root, filename))
    media_paths = [item for item in media_paths if item.split('.')[-1].lower() in extension_list]
    return media_paths


html_paths = find_paths('./', ['html','php','htm'])

html_paths = sorted(list(set(html_paths)))

len(html_paths)

In [ ]:
#%%time

## Checking all links

import csv

os.chdir(epc_pathname)

header = ['url', 'linked from', 'server response', 'redirect link']

error_pages = []
broken_links = []

repo_pathname = '/media/smcl/Audio/epc_pages/'

counter = 0

with open(os.path.join(repo_pathname, 'EPC_Link_Status_Table.csv'), 'w') as fo:
    csv_writer = csv.writer(fo)
    csv_writer.writerow(header)

for html_path in html_paths:
    if counter%50 == 0:
        print(counter)
    counter+=1
    try:
        page = open(html_path).read()
    except:
        try:
            page = page.decode('utf8')
        except:
            error_pages.append(html_path)
    #except: pass
    try:
        soup = BeautifulSoup(page, 'lxml')
        for link in soup.find_all('a'):
            url = link['href']
            if url[:5] == '/epc/':
                url = url.replace('/epc/', 'https://writing.upenn.edu/epc/')
            if url[:6] == './epc/':
                url = url.replace('./epc/', 'https://writing.upenn.edu/epc/')
            if 'http' in url:
                time.sleep(0.03)
                redirect_link = ''
                r = ''
                try:
                    r = requests.head(url)
                    if r.status_code in (301, 302, 303, 304):
                        try:
                            redirect_link = r.headers['Location'] ######## <-------------------
                        except: 
                            pass
                    row = [url, html_path.replace('./epc/','/epc/'), str(r.status_code), redirect_link]
                    with open(os.path.join(repo_pathname, 'EPC_Link_Status_Table.csv'), 'a') as fo:
                        csv_writer = csv.writer(fo)
                        csv_writer.writerow(row)
                except requests.ConnectionError as e:
                    broken_links.append(url)
                    row = [url, html_path.replace('./epc/','/epc/'), 'dead', '']
                    csv_writer.writerow(row)
                time.sleep(0.01)
    except Exception as e:
        error_pages.append(html_path)

